# SlideSeq Spatial Transcriptomics analysis of Muscle Regeneration 

As described at:
Integrated single-cell and spatial transcriptomic analysis of skeletal muscle regeneration 
Lauren Walter , Jessica Orton , E. H. Hannah Fong , Ms Viviana Maymi , Dr Brian Rudd , Dr Jennifer Elisseeff , Ioannis Ntekas , Iwijn De Vlaminck, Benjamin D. Cosgrove* 


ST analysis by Ioannis Ntekas 

email: in68@cornell.edu

## Import Packages


In [ ]:
import squidpy as sq
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import os
import seaborn as sns
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patches as mpatches
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import scipy.spatial as scisp
from matplotlib.colors import ListedColormap
from scipy.stats import ttest_ind, mannwhitneyu, ranksums



import gc


###import functions from ST_utils.py
import ST_utils

gc.enable()

In [ ]:
sc.settings.verbosity = 3  
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(5, 5),vector_friendly=True, dpi_save=300)
# Font properties for the scale bar
fontprops = fm.FontProperties(size=10)
# Set global figure parameters for scanpy
sc.settings.set_figure_params(fontsize=7)

In [ ]:
# Define colormap to use with light grey as the zero/ND value 
viridis = mpl.colormaps['viridis_r'].resampled(256)
newcolors = viridis(np.linspace(0, 1, 256))
light_grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = light_grey
newcmp = ListedColormap(newcolors)


# Define colormap to use with light grey as the zero/ND value 
Reds = mpl.colormaps['Reds'].resampled(256)
newcolors = Reds(np.linspace(0, 1, 256))
light_grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = light_grey
newcmp2 = ListedColormap(newcolors)

# Define colormap to use with light grey as the zero/ND value 
Magma = mpl.colormaps['magma'].resampled(256)
newcolors = Magma(np.linspace(0, 1, 256))
light_grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = light_grey
newcmp3 = ListedColormap(newcolors)


# Define colormap to use with light grey as the zero/ND value 
inferno = mpl.colormaps['inferno'].resampled(256)
newcolors = inferno(np.linspace(0, 1, 256))
light_grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = light_grey
newcmp4 = ListedColormap(newcolors)

In [ ]:
# Load the Young and Geriatric object on a dictionary for easier handling 0:Y, 1:G

adata_dict = {}
Sample_list = [11,12]

for i in[0,1]:
    temp = sc.read_10x_mtx(f"/fs/cbsuvlaminck2/workdir/in68/Muscle/out/Sample{Sample_list[i]}/STARsolo/Solo.out/GeneFull/raw/")
    temp.write_h5ad(f"/fs/cbsuvlaminck2/workdir/in68/Muscle/out/Sample{Sample_list[i]}/STARsolo/Solo.out/GeneFull/raw/Sample_{Sample_list[i]}.h5ad")
    temp.raw = temp.copy()
    sc.pp.filter_cells(temp, min_counts = 20)
    # add cell2location prediction scores and related vars to the object --copy the .obsm, .uns 
    adata =sc.read_h5ad (f"/workdir/in68/Muscle/cell2location/cell2location_out/Muscle_{Sample_list[i]}_0301.h5ad")
    temp.obsm = adata.obsm
    temp.uns = adata.uns
    adata_dict[i] = temp
    adata_dict[i].var_names_make_unique()

    
    adata_dict[i].obs[adata_dict[i].uns['mod']['factor_names']] = adata_dict[i].obsm['q05_cell_abundance_w_sf']
    adata_dict[i].obsm['q05_cell_proportions'] = adata_dict[i].obsm['q05_cell_abundance_w_sf'].div(adata_dict[i].obsm['q05_cell_abundance_w_sf'].sum(axis=1), axis=0)

    adata_dict[i].obs["total_abundance"] = adata_dict[i].obsm['q05_cell_abundance_w_sf'].sum(axis = 1)
    # adata_dict[i].obs[adata_dict[i].uns['mod']['factor_names']] = adata_dict[i].obsm['q05_cell_abundance_w_sf']
    adata_dict[i].obs[adata_dict[i].uns['mod']['factor_names']] = adata_dict[i].obsm['q05_cell_proportions']
    adata_dict[i].obs["max_pred"] = adata_dict[i].obs[adata_dict[i].uns['mod']['factor_names']].max(axis=1)
    #Set max_pred_celltype as the dominant celltype based on cell2loc results
    adata_dict[i].obs["max_pred_celltype"] = adata_dict[i].obs[adata_dict[i].uns['mod']['factor_names']].idxmax(axis=1)
    adata_dict[i].obs["max_pred_celltype"] = adata_dict[i].obs["max_pred_celltype"].astype('category')




    #####Add spatial coordinates and then filer cells with less that 10 neighbors at 100um
    # Add spatial coords
    adata_dict[i] = add_spatial_coordinates(adata_dict[i], f"/fs/cbsuvlaminck2/workdir/in68/Utils/Curio_BB/A0018_0{Sample_list[i]}_BeadBarcodes.txt")
    # Convert distances to um
    adata_dict[i].obsm['spatial'] = adata_dict[i].obsm['spatial'] /1.54
    # Calculate euclidean distances
    adata_dict[i].obsm["spatial_distances"] = scisp.distance.squareform(scisp.distance.pdist(adata_dict[i].obsm['spatial']))
    # Filter smear/singlets
    adata_dict[i].obs["spatial_neighbors_100_true"] = np.sum(adata_dict[i].obsm["spatial_distances"] < 100, axis=0)
    adata_dict[i] = adata_dict[i][adata_dict[i].obs["spatial_neighbors_100_true"] > 10]
    
    # QC and filtering
    sc.pp.filter_genes(adata_dict[i], min_cells = 10)

    adata_dict[i].var["mito"] = adata_dict[i].var_names.str.startswith("Mt-")
    sc.pp.calculate_qc_metrics(adata_dict[i], qc_vars=["mito"], inplace=True)

    # Normalization
    sc.pp.normalize_total(
            adata_dict[i], 
            target_sum=1e4,
        inplace=True)
    sc.pp.log1p(adata_dict[i])
    
del temp 
 

### Define injury loci
Injury locus deffinition based on immune cell density and location

In [ ]:
# Define injury locus based on the kNN approach. Calculate immune neighbors 

for adata in adata_dict.values():

    for celltype in np.unique(adata.obs['max_pred_celltype']):
        adata = calculate_celltype_neighbors(adata, 300, celltype, 'max_pred_celltype')
    
    adata.obs['Immune_neighbors'] = adata.obs['T cells_neighbors'] + adata.obs['B cells_neighbors'] + adata.obs['NK cells_neighbors'] + adata.obs['Dendritic cells_neighbors'] + adata.obs['Neutrophils_neighbors'] + adata.obs['Monocytes/Macrophages_neighbors']
gc.collect()   

In [ ]:
# Geriatric Muscle

mask = (adata_dict[1].obs["Immune_neighbors"] > 20) & (adata_dict[1].obsm["spatial"][:,0] > 1945)
adata_dict[1].obs['injury_classification'] = ''
idx = adata_dict[1][mask].obs.index
idx_rev = adata_dict[1][~mask].obs.index
adata_dict[1].obs.loc[idx_rev, 'injury_classification'] = 'outside'
adata_dict[1].obs.loc[idx, 'injury_classification'] = 'injury'

sq.pl.spatial_scatter(adata_dict[1],color ="injury_classification", size= 1, shape=None,figsize= [5,5],cmap=newcmp,vmin = 0.4)

In [ ]:
# Young Muscle 

mask = (adata_dict[0].obs["Immune_neighbors"] > 20) & (adata_dict[0].obsm["spatial"][:,1] < 1500)
adata_dict[0].obs['injury_classification'] = ''
idx = adata_dict[0][mask].obs.index
idx_rev = adata_dict[0][~mask].obs.index
adata_dict[0].obs.loc[idx_rev, 'injury_classification'] = 'outside'
adata_dict[0].obs.loc[idx, 'injury_classification'] = 'injury'

sq.pl.spatial_scatter(adata_dict[0],color ="injury_classification", size= 0.1, shape=None,figsize= [5,5],cmap=newcmp,vmin = 0.4)

In [ ]:
##Discard singlets/small islands
for adata in adata_dict.values():
    adata= calculate_celltype_neighbors(adata, 20, celltype_label='injury',cell_obs='injury_classification')
    mask = adata.obs["injury_neighbors"] > 10
    idx = adata[mask].obs.index
    idx_rev = adata[~mask].obs.index
    adata.obs.loc[idx_rev, 'injury_classification'] = 'outside'
    adata.obs.loc[idx, 'injury_classification'] = 'injury'
    
sq.pl.spatial_scatter(adata_dict[0],color ="injury_classification", size= 1, shape=None,figsize= [5,5],cmap=newcmp,vmin = 0.4)
sq.pl.spatial_scatter(adata_dict[1],color ="injury_classification", size= 1, shape=None,figsize= [5,5],cmap=newcmp,vmin = 0.4)

### Define cell type and injury locus colors 

In [ ]:
celltype_colors = [
    '#F20A53',
    '#9C6DA5',
    '#2E3B65',
    '#3C0AAA',
    '#8BA690',
    '#067F71',
    '#891B5A',
    '#FA78FA',
    '#C85A00',
    '#076594',
    '#709EB5',
    '#826E00',
    '#5078FA',
    '#AFE4DE',
    '#793EA8'
]  
loc_colors =  ['#EC4F38','#2B8076']




 
adata_dict[0].uns['max_pred_celltype_colors'] = celltype_colors
adata_dict[0].uns['loc_colors'] = loc_colors
adata_dict[1].uns['max_pred_celltype_colors'] = celltype_colors
adata_dict[1].uns['loc_colors'] = loc_colors

### Make barplots for cell abundances in spatial data 

In [ ]:
###Store normalization info as uns
for i in[0,1]:
    total_beads = len(adata_dict[i].obs)
    injury = len(adata_dict[i][adata_dict[i].obs['injury_classification'] == 'injury' ].obs)
    outside = len(adata_dict[i][adata_dict[i].obs['injury_classification'] != 'injury' ].obs)
    
    adata_dict[i].uns['total_beads'] = total_beads
    adata_dict[i].uns['injury'] = injury
    adata_dict[i].uns['outside'] = outside


In [ ]:
os.chdir('/workdir/in68/Muscle/figs/')

In [ ]:
# Custom font settings
plt.rcParams.update({'font.size': 7})  # Set global font size

# Function to format the y-axis ticks as percentages
def to_percentage(x, pos):
    return f'{int(x)}%'

formatter = FuncFormatter(to_percentage)

# Define the subset of cell types to include in the plot in the specified order
cell_types_subset = ['MuSCs', 'Fusing Myocytes', 'Monocytes/Macrophages', 'B cells', 'T cells', 'Dendritic cells', 'FAPs', 'Endothelial cells']

# Create a figure with subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6.2, 2.5), dpi=300)  # Total size of the figure
fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.15)  # Adjust space to optimize layout

for i, location in enumerate(np.unique(adata_dict[0].obs['injury_classification'])):
    category = location
    colors = ['#F08205', '#4C026E']
    dataset_names = ['Young', 'Geriatric']
    
    cell_type_counts = {}
    for idx, (key, adata) in enumerate(adata_dict.items()):
        adata = adata[adata.obs['injury_classification'] == category].copy()
        counts = adata.obs['max_pred_celltype'].value_counts().rename(dataset_names[idx]).filter(items=cell_types_subset)
        cell_type_counts[dataset_names[idx]] = counts / adata.uns[category] * 100
    
    # Convert the counts to a DataFrame for easier plotting
    df_counts = pd.DataFrame(cell_type_counts).fillna(0).reindex(cell_types_subset).reset_index()
    df_counts = df_counts.rename(columns={'index': 'max_pred_celltype'}).melt(id_vars='max_pred_celltype', var_name='Dataset', value_name='Count')
    
    # Plotting
    ax = axes[i]
    sns.barplot(data=df_counts, x='max_pred_celltype', y='Count', hue='Dataset', palette=colors, edgecolor=None, dodge=True, ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
    ax.set_ylim(0, 20)  # Setting a consistent y-axis limit
    ax.yaxis.set_major_formatter(formatter)  # Set y-axis labels to show percentages

    # Explicit font size settings for elements
    ax.tick_params(axis='both', which='major', labelsize=7)
    ax.set_xlabel(ax.get_xlabel(), fontsize=7)
    ax.set_ylabel(ax.get_ylabel(), fontsize=7)
    ax.legend_.remove()  # Remove the legend
    ax.set_xlabel('')  # Remove x-axis label

    if i == 1:
        ax.set_ylabel('')  # Remove y-axis label for the second plot
        ax.set_yticklabels([])  # Remove y-axis tick labels for the second plot
        

# Adjust layout and save
plt.tight_layout()  # Auto-adjust layout
#plt.savefig('Cell_Type_Comparison_1.pdf', bbox_inches='tight', dpi=300)  # Save with tight bounding box
plt.show()  # Show the plot


In [ ]:
formatter = FuncFormatter(to_percentage)
# Define the subset of cell types to include in the plot in the specified order
cell_types_subset = ['Myonuclei']

# Create a figure with subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(1.5, 2.5), dpi=300)  # Narrower figure as less content needs to be displayed
fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.4)  # Adjust space to optimize layout

for i, location in enumerate(np.unique(adata_dict[0].obs['injury_classification'])):
    category = location
    colors = ['#F08205', '#4C026E']
    dataset_names = ['Young', 'Geriatric']
    
    cell_type_counts = {}
    for idx, (key, adata) in enumerate(adata_dict.items()):
        adata = adata[adata.obs['injury_classification'] == category].copy()
        counts = adata.obs['max_pred_celltype'].value_counts().rename(dataset_names[idx]).filter(items=cell_types_subset)
        cell_type_counts[dataset_names[idx]] = counts / adata.uns[category] * 100
    
    # Convert the counts to a DataFrame for easier plotting
    df_counts = pd.DataFrame(cell_type_counts).fillna(0).reindex(cell_types_subset).reset_index()
    df_counts = df_counts.rename(columns={'index': 'max_pred_celltype'}).melt(id_vars='max_pred_celltype', var_name='Dataset', value_name='Count')
    
    # Plotting
    ax = axes[i]
    sns.barplot(data=df_counts, x='max_pred_celltype', y='Count', hue='Dataset', palette=colors, edgecolor=None, dodge=True, ax=ax)
    ax.set_xticklabels(['Myonuclei'], rotation=90, ha='right')  # Use long placeholder to ensure space is similar
    ax.set_ylim(0, 100)  # Setting a consistent y-axis limit
    ax.yaxis.set_major_formatter(formatter)  # Set y-axis labels to show percentages
    
    ax.legend_.remove()  # Remove the legend
    
    ax.tick_params(axis='both', which='major', labelsize=7)
    ax.set_xlabel('')  # Remove x-axis label
    ax.set_ylabel(ax.get_ylabel(), fontsize=7)
    
    if i == 1:
        ax.set_ylabel('')  # Remove y-axis label for the second plot
        ax.set_yticklabels([])  # Remove y-axis tick labels for the second plot

# Adjust layout and save
plt.tight_layout()  # Auto-adjust layout
#plt.savefig('Cell_Type_Comparison_2.pdf', bbox_inches='tight', dpi=300)  # Save with tight bounding box
plt.show()  # Show the plot


### Calculate senescence score 

In [ ]:
# Load senescence score genes 
senescence = pd.read_csv('/workdir/in68/Muscle/metadata/FBR_Gene_List.csv')

gene_list = senescence[senescence['logFC'] > 0]['Gene (mouse)'].astype('str')
print(gene_list)



### Filter gene list 
muscs = {}
i = 0
for adata in adata_dict.values():
    muscs[i] = adata[adata.obs['max_pred_celltype'] == 'MuSCs']
    i = i+1
sc.pp.filter_genes(muscs[0],min_cells= 2)
sc.pp.filter_genes(muscs[1],min_cells= 2)

common_genes = np.intersect1d(muscs[1].var_names.astype('str'),muscs[0].var_names.astype('str'))

score_genes = np.intersect1d(common_genes,gene_list)


sc.tl.score_genes(adata_dict[0],gene_list= score_genes, score_name= 'Senescence_score',use_raw=False,n_bins= 1000,ctrl_size=50)
sc.tl.score_genes(adata_dict[1],gene_list= score_genes, score_name= 'Senescence_score',use_raw=False, n_bins= 1000, ctrl_size=50)


In [ ]:
# Define groups for comparison
groups = {
    'Young_m': (adata_dict[0], adata_dict[0].obs['max_pred_celltype'] == 'MuSCs', 'Young', 'MuSCs_all'),
    'Geriatric_m': (adata_dict[1], adata_dict[1].obs['max_pred_celltype'] == 'MuSCs', 'Geriatric', 'MuSCs_all'),
    'Young_m_injury': (adata_dict[0], (adata_dict[0].obs['max_pred_celltype'] == 'MuSCs') & (adata_dict[0].obs['injury_classification'] == 'injury'), 'Young', 'MuSCs_injury'),
    'Geriatric_m_injury': (adata_dict[1], (adata_dict[1].obs['max_pred_celltype'] == 'MuSCs') & (adata_dict[1].obs['injury_classification'] == 'injury'), 'Geriatric', 'MuSCs_injury'),
    'Young_m_noninjury': (adata_dict[0], (adata_dict[0].obs['max_pred_celltype'] == 'MuSCs') & (adata_dict[0].obs['injury_classification'] != 'injury'), 'Young', 'MuSCs_noninjury'),
    'Geriatric_m_noninjury': (adata_dict[1], (adata_dict[1].obs['max_pred_celltype'] == 'MuSCs') & (adata_dict[1].obs['injury_classification'] != 'injury'), 'Geriatric', 'MuSCs_noninjury'),
    'Young_non': (adata_dict[0], adata_dict[0].obs['max_pred_celltype'] != 'MuSCs', 'Young', 'non_MuSCs_all'),
    'Geriatric_non': (adata_dict[1], adata_dict[1].obs['max_pred_celltype'] != 'MuSCs', 'Geriatric', 'non_MuSCs_all'),
    'Young_non_injury': (adata_dict[0], (adata_dict[0].obs['max_pred_celltype'] != 'MuSCs') & (adata_dict[0].obs['injury_classification'] == 'injury'), 'Young', 'non_MuSCs_injury'),
    'Geriatric_non_injury': (adata_dict[1], (adata_dict[1].obs['max_pred_celltype'] != 'MuSCs') & (adata_dict[1].obs['injury_classification'] == 'injury'), 'Geriatric', 'non_MuSCs_injury'),
    'Young_non_noninjury': (adata_dict[0], (adata_dict[0].obs['max_pred_celltype'] != 'MuSCs') & (adata_dict[0].obs['injury_classification'] != 'injury'), 'Young', 'non_MuSCs_noninjury'),
    'Geriatric_non_noninjury': (adata_dict[1], (adata_dict[1].obs['max_pred_celltype'] != 'MuSCs') & (adata_dict[1].obs['injury_classification'] != 'injury'), 'Geriatric', 'non_MuSCs_noninjury')
}

# Extract data and compare each Young vs Geriatric group
final_data = []
comparisons = [
    ('Young_m', 'Geriatric_m'),
    ('Young_m_injury', 'Geriatric_m_injury'),
    ('Young_m_noninjury', 'Geriatric_m_noninjury'),
    ('Young_non', 'Geriatric_non'),
    ('Young_non_injury', 'Geriatric_non_injury'),
    ('Young_non_noninjury', 'Geriatric_non_noninjury')
]

# Loop through each comparison, extract data, plot, and calculate statistics
for group1, group2 in comparisons:
    df1 = extract_data(*groups[group1])
    df2 = extract_data(*groups[group2])
    final_data.append(df1)
    final_data.append(df2)
    
    print(f"\nComparing {group1} vs {group2}")
    # print(f"{group1}: Mean = {np.mean(df1['Senescence_score'])}, Median = {np.median(df1['Senescence_score'])}, N = {len(df1)}")
    # print(f"{group2}: Mean = {np.mean(df2['Senescence_score'])}, Median = {np.median(df2['Senescence_score'])}, N = {len(df2)}")
    
    # plot_boxplot_for_arrays(df1['Senescence_score'], df2['Senescence_score'], array_1=group1, array_2=group2)
    
    results = compare_datasets(df1['Senescence_score'], df2['Senescence_score'], n_comparisons=1)
    for test, value in results.items():
        print(f"{test}: {value}")

# Combine all data into a single DataFrame
final_df = pd.concat(final_data)

# Reorder columns
final_df = final_df[['cell_name', 'age', 'location', 'Senescence_score']]

# Export to CSV
final_df.to_csv('senescence_scores.csv', index=False)

print("Data has been written to senescence_scores.csv")


In [ ]:
# Define the colors
colors = ['#F08205', '#4C026E']

# Create the seaborn box plot with specified colors and adjust box width and outlier size
ax = sns.boxplot(data=final_df, x="location", y='Senescence_score', hue="age", palette=colors, width=0.5, fliersize=2)

# Make outliers slightly transparent
for artist in ax.artists:
    artist.set_edgecolor('black')  # Set outlier edge color to black
    for line in artist.get_lines():
        line.set_alpha(0.1)  # Set outlier transparency to 0.5

# Remove legends
#plt.legend().remove()

# Set figure size
plt.gcf().set_size_inches(10, 5)

# Customize y-axis display
plt.yticks(fontsize=7)
plt.gca().set_ylim([-0.65, 1.05])  # Set the y-axis limits
plt.gca().set_yticks([-0.6,-0.4,-0.2, 0, 0.2, 0.4, 0.6, 0.8,1.0])  # Set y-axis ticks

#Give angle to x labels
plt.xticks(rotation=45, ha='right', fontsize=7)
# Save the figure as PDF with high resolution
#plt.savefig("box_plot.pdf", dpi=300, bbox_inches='tight')

# Show the plot (optional)
plt.show()


In [ ]:
### Save source data for figure 

### Full size spatial plots

In [ ]:
titles = ['Young', 'Geriatric']

plot_spatial_data(adata_dict, obs=['max_pred_celltype'], titles=titles,  spot_size=15)


In [ ]:
titles = ['Young', 'Geriatric']

plot_spatial_data(adata_dict, obs=['injury_classification'], titles=titles,  spot_size=15)


### Zoomed-in plots

In [ ]:


crop = {0: [1500, 2800, 2000, 700], 1: [1500, 2800, 2800, 1500]}
titles = ['Young', 'Geriatric']

plot_spatial_data(adata_dict, obs=['max_pred_celltype'], titles=titles, crop=crop, spot_size=15)


In [ ]:
crop = {0: [1500, 2800, 2000, 700], 1: [1500, 2800, 2800, 1500]}
titles = ['Young', 'Geriatric']
plot_spatial_data(adata_dict, obs=['injury_classification'], titles=titles, crop=crop,spot_size=15)


In [ ]:

crop = {0: [1500, 2800, 2000, 700], 1: [1500, 2800, 2800, 1500]}
titles = ['Young', 'Geriatric']
plot_spatial_data(adata_dict, obs=['Senescence_score'], titles=titles, crop=crop, cmap=newcmp4, vmin=0.0000001, vmax=0.8, spot_size=15)


In [ ]:
adata_dict[0].write_h5ad('/fs/cbsuvlaminck2/workdir/in68/Muscle/out/Sample11/Young_TA_ST_5dpi_pp.h5ad')
adata_dict[1].write_h5ad('/fs/cbsuvlaminck2/workdir/in68/Muscle/out/Sample12/Geriatric_TA_ST_5dpi_pp.h5ad')